<a href="https://colab.research.google.com/github/icervecon/PythonAssetMagement/blob/main/PE_4_04_Python_AM_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical Exercise 4.04: Inmunization

In [12]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Initial data

required_payment = 5000000  # Future value in year 3
target_duration = 3

# Bond durations

duration_A = 1.97
duration_B = 3.80
duration_C = 4.63

# Current bond prices

price_A = 1000
price_B = 1000
price_C = 1000

# Bond Yields (discount rates)

y_A = 0.03
y_B = 0.035
y_C = 0.04

# Define the equations to be solved

def duration_matching_minimize(weights):
    w_A, w_B, w_C = weights


    # The duration of the portfolio should be 3 years.
    duration_constraint = (w_A * duration_A + w_B * duration_B + w_C * duration_C - 3)**2

    # Weights must add up to 100%.

    weights_sum_constraint = (w_A + w_B + w_C - 1)**2
    return duration_constraint + weights_sum_constraint

# Restrictions to ensure that weights are >= 0

constraints = (
    {'type': 'ineq', 'fun': lambda x: x[0]},  # w_A >= 0
    {'type': 'ineq', 'fun': lambda x: x[1]},  # w_B >= 0
    {'type': 'ineq', 'fun': lambda x: x[2]}   # w_C >= 0
)

# Initial assumptions for bond weights A, B and C

initial_weights = [0.33, 0.33, 0.34]

# Solve the equations to find the correct bond weights.

result = minimize(duration_matching_minimize, initial_weights, constraints=constraints)
weights_solution = result.x

# Bond weights

w_A, w_B, w_C = weights_solution


# Calculate the IRR of the portfolio as the weighted average of the bond IRRs.

portfolio_yield = w_A * y_A + w_B * y_B + w_C * y_C

# Calculate the present value of the liability using the IRR of the portfolio.

def present_value(rate, future_value, years):
    return future_value / (1 + rate)**years

present_value_liability = present_value(portfolio_yield, required_payment, 3)

# Calculate the amount to invest in each bond

amount_invested_A = w_A * present_value_liability
amount_invested_B = w_B * present_value_liability
amount_invested_C = w_C * present_value_liability

# Calculate the number of units of each bond

no_securities_A = amount_invested_A / price_A
no_securities_B = amount_invested_B / price_B
no_securities_C = amount_invested_C / price_C

# Create the results DataFrame

results = pd.DataFrame({
    'Bond': ['A', 'B', 'C'],
    'Weight': [w_A, w_B, w_C],
    'Amount Invested': [amount_invested_A, amount_invested_B, amount_invested_C],
    'Current Price': [price_A, price_B, price_C],
    'Number of Securities': [round(no_securities_A,0), round(no_securities_B,0), round(no_securities_C,0)],
    'Bond Yield(%)': [y_A*100, y_B*100, y_C*100],
    'Duration': [duration_A, duration_B, duration_C]
})

# Round IRR Bond to four decimal places

results['Bond Yield(%)'] = results['Bond Yield(%)'].apply(lambda x: '{:.2f}'.format(x))

# Add additional portfolio details

portfolio_yield_rounded = '{:.2f}'.format(portfolio_yield*100)
present_value_liability_rounded = round(present_value_liability, 2)

# Create an additional row for portfolio details

portfolio_details = pd.DataFrame({
    'Bond': ['Inmunized Portfolio'],
    'Weight': [1.0],
    'Amount Invested': [present_value_liability_rounded],
    'Current Price': ['-'],
    'Number of Securities': ['-'],
    'Bond Yield(%)': [portfolio_yield_rounded],
    'Duration': [target_duration]
})

# Concatenate the results table with the portfolio details row

final_results= pd.concat([results, portfolio_details], ignore_index=True)

# Display results without scientific notation and with thousands separator

pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))
final_results


,Bond,Weight,Amount Invested,Current Price,Number of Securities,Bond Yield(%),Duration
0,A,0.52,"2,371,080.88",1000,"2,371.00",3.00,1.97
1,B,0.29,"1,300,479.59",1000,"1,300.00",3.50,3.80
2,C,0.19,"860,017.53",1000,860.00,4.00,4.63
3,Inmunized Portfolio,1.00,"4,531,577.71",-,-,3.33,3.00


Now let's check that immunization is effective by simulating changes in interest rates.

In [14]:
# Parallel shift test (stress-test immunization).
# We are going to bump all discount rates up and down by the same amount and reprice the asset portfolio and the liability (5M due at year 3)

# Inputs from before
nA = int(round(no_securities_A))
nB = int(round(no_securities_B))
nC = int(round(no_securities_C))

# liability
FV_liab = float(required_payment)
T_liab = 3

# cash-flow schedules (year -> CF)
cf_A = {1: 30, 2: 1030}
cf_B = {1: 35, 2: 35, 3: 35, 4: 1035}
cf_C = {1: 40, 2: 40, 3: 40, 4: 40, 5: 1040}


def pv(fv, r, t):         # annual compounding
    return fv / (1 + r) ** t

def price_from_cf(cf_dict, y):
    return sum(cf / (1 + y) ** t for t, cf in cf_dict.items())

def price_portfolio(y_shift):
    pA = price_from_cf(cf_A, y_A + y_shift)
    pB = price_from_cf(cf_B, y_B + y_shift)
    pC = price_from_cf(cf_C, y_C + y_shift)
    return nA * pA + nB * pB + nC * pC

def pv_liability(y_shift):
    return pv(FV_liab, portfolio_yield + y_shift, T_liab)

def horizon_value_one_bond(cf_dict, y, shift, units, horizon=3):
    y_s = y + shift
    # cashflows up to horizon are reinvested to horizon
    before = sum(cf * (1 + y_s) ** (horizon - t) for t, cf in cf_dict.items() if t <= horizon)
    # remaining cashflows are priced at horizon
    after_price = sum(cf / (1 + y_s) ** (t - horizon) for t, cf in cf_dict.items() if t > horizon)
    return units * (before + after_price)

def horizon_value_assets(shift):
    return (
        horizon_value_one_bond(cf_A, y_A, shift, nA) +
        horizon_value_one_bond(cf_B, y_B, shift, nB) +
        horizon_value_one_bond(cf_C, y_C, shift, nC)
    )

# run parallel shifts (±200 bps in 25-bp steps)
bps_grid = np.arange(-200, 201, 25)
shifts = bps_grid / 10_000

# PV at t0 test
pv_rows = []
for s in shifts:
    pv_rows.append({
        "Shift (bps)": int(s * 10_000),
        "PV Assets": price_portfolio(s),
        "PV Liability": pv_liability(s)
    })
pv_df = pd.DataFrame(pv_rows)
pv_df["PV Surplus"] = pv_df["PV Assets"] - pv_df["PV Liability"]

# Horizon-value (t=3) test
hv_rows = []
for s in shifts:
    hv_rows.append({
        "Shift (bps)": int(s * 10_000),
        "Horizon Asset Value": horizon_value_assets(s),
        "Liability at t=3": float(FV_liab)
    })
hv_df = pd.DataFrame(hv_rows)
hv_df["Horizon Surplus"] = hv_df["Horizon Asset Value"] - hv_df["Liability at t=3"]

# Worst-case deficits and minimum cash cushion today (for the horizon test)
worst_hv_deficit_t3 = max(0.0, -hv_df["Horizon Surplus"].min())     # need at t=3
min_cushion_today = worst_hv_deficit_t3 / (1 + port_yield) ** T_liab # deposit today

# prints
print("PV stress (t=0):")
print(pv_df.to_string(index=False))
print("\nHorizon stress (t=3):")
print(hv_df.to_string(index=False))
print(f"\nMinimum cash cushion today to avoid any horizon shortfall: €{min_cushion_today:,.2f}")


PV stress (t=0):
 Shift (bps)    PV Assets  PV Liability  PV Surplus
        -200 4,805,721.64  4,805,225.55      496.09
        -175 4,770,034.27  4,769,835.30      198.97
        -150 4,734,742.89  4,734,791.73      -48.84
        -125 4,699,841.84  4,700,090.61     -248.77
        -100 4,665,325.56  4,665,727.76     -402.20
         -75 4,631,188.56  4,631,699.06     -510.50
         -50 4,597,425.48  4,598,000.48     -575.00
         -25 4,564,031.02  4,564,628.00     -596.98
           0 4,531,000.00  4,531,577.71     -577.71
          25 4,498,327.30  4,498,845.72     -518.42
          50 4,466,007.91  4,466,428.20     -420.29
          75 4,434,036.88  4,434,321.39     -284.52
         100 4,402,409.36  4,402,521.58     -112.22
         125 4,371,120.59  4,371,025.11       95.48
         150 4,340,165.87  4,339,828.36      337.51
         175 4,309,540.58  4,308,927.78      612.80
         200 4,279,240.20  4,278,319.87      920.33

Horizon stress (t=3):
 Shift (bps)  Horizon As